### Preferences model

In [1]:
import pysmile
import pysmile_license
import numpy as np
import pandas as pd

from network_functions import calculate_network_utilities, new_screening_strategy, old_screening_strategy, create_folders_logger
from simulations import plot_classification_results
from plots import plot_estimations_w_error_bars, plot_screening_counts

from preprocessing import preprocessing
from concurrent.futures import ProcessPoolExecutor, as_completed
from tqdm import tqdm
import matplotlib.pyplot as plt

import yaml
with open('config.yaml', 'r') as file:
    cfg = yaml.safe_load(file)

import argparse
import pdb
import matplotlib
matplotlib.use('Agg')

import logging
import datetime 
import os

In [2]:
cfg['max_workers']

8

In [3]:
cfg['rel_point_cond_mut_info']

{'PE_cost': 50, 'PE_info': 4.1, 'PE_prob': 0.7}

In [4]:
# from main import main
from use_case_new_strategy import use_case_new_strategy
from full_example import full_example
from update import update_influence_diagram
from network_functions import create_folders_logger
from sens_analysis_elicitation import sens_analysis_elicitation
from sens_analysis_PE_method import sens_analysis_PE_method

In [5]:
cfg["patient_chars"]

{'Age': 'age_4_adult',
 'Sex': 'M',
 'SD': 'SD_2_normal',
 'PA': 'PA_2',
 'Smoking': 'sm_1_not_smoker',
 'BMI': 'bmi_2_normal',
 'Alcohol': 'low'}

## Use cases

### Individual cases

**Benchmark patient**

In [7]:
label = 'notebook_benchmark_patient'
logger, log_dir = create_folders_logger(label = label, output_dir = 'notebook/' + label, date = False, time = False)

update_influence_diagram(
    model_type = cfg["model_type"],
    value_function = cfg["value_function"],
    elicit = cfg["elicit"],
    noise = cfg["noise"],
    calculate_info_values= cfg["calculate_info_values"],
    ref_patient_chars = cfg["patient_chars"],
    new_test = cfg["new_test"],
    logger = logger,
    output_dir = 'notebook/' + label
)

2025-01-29 11:16:56,243 - INFO - Model type: linear
2025-01-29 11:16:56,244 - INFO - Reading network...
2025-01-29 11:16:56,285 - INFO - Calculating information values...
2025-01-29 11:17:54,365 - INFO - Eliciting value of comfort...
2025-01-29 11:17:54,366 - INFO - No elicitation of lambda values, taking default values...
2025-01-29 11:17:54,368 - INFO - Lambda values: [7.         4.01194248 6.80198452 4.01194248 6.80198452 4.01194248
 6.80198452 4.01194248 6.80198452 4.01194248 4.16848007 4.01194248
 4.16848007 4.01194248]
2025-01-29 11:17:54,368 - INFO - Saving network...
2025-01-29 11:17:54,406 - INFO - Plotting info functions...
2025-01-29 11:18:29,015 - INFO - Calculating final utilities...
2025-01-29 11:18:29,023 - INFO - Best info: 15.299634978175163
2025-01-29 11:18:29,023 - INFO - Worst info: 4.450147717014403e-308
2025-01-29 11:18:29,024 - INFO - PE info: 4.1
2025-01-29 11:18:29,025 - INFO - v_best: 104.06788025218766
2025-01-29 11:18:29,025 - INFO - v_worst: -3.910235286299

**Benchmark patient risk seeking**

In [8]:
label = 'notebook_benchmark_risk_seeking'
logger, log_dir = create_folders_logger(label = label, output_dir = 'notebook/' + label, date = False, time = False)

update_influence_diagram(
    model_type = cfg["model_type"],
    value_function = cfg["value_function"],
    elicit = cfg["elicit"],
    noise = cfg["noise"],
    calculate_info_values= cfg["calculate_info_values"],
    ref_patient_chars = cfg["patient_chars"],
    new_test = cfg["new_test"],
    logger = logger,
    output_dir = 'notebook/' + label,
    change_risk_param=True,
    rho_param= 0.005
)

2025-01-29 11:18:58,166 - INFO - Model type: linear
2025-01-29 11:18:58,166 - INFO - Model type: linear
2025-01-29 11:18:58,167 - INFO - Reading network...
2025-01-29 11:18:58,167 - INFO - Reading network...
2025-01-29 11:18:58,181 - INFO - Calculating information values...
2025-01-29 11:18:58,181 - INFO - Calculating information values...
2025-01-29 11:19:58,855 - INFO - Eliciting value of comfort...
2025-01-29 11:19:58,855 - INFO - Eliciting value of comfort...
2025-01-29 11:19:58,857 - INFO - No elicitation of lambda values, taking default values...
2025-01-29 11:19:58,857 - INFO - No elicitation of lambda values, taking default values...
2025-01-29 11:19:58,859 - INFO - Lambda values: [7.         4.01194248 6.80198452 4.01194248 6.80198452 4.01194248
 6.80198452 4.01194248 6.80198452 4.01194248 4.16848007 4.01194248
 4.16848007 4.01194248]
2025-01-29 11:19:58,859 - INFO - Lambda values: [7.         4.01194248 6.80198452 4.01194248 6.80198452 4.01194248
 6.80198452 4.01194248 6.8019

**Benchmark patient added evidence**

In [9]:
label = 'notebook_benchmark_addded_ev'
logger, log_dir = create_folders_logger(label = label, output_dir = 'notebook/' + label, date = False, time = False)

patient_chars = cfg["patient_chars"].copy()
patient_chars["Diabetes"] = True
patient_chars["Hypertension"] = True

update_influence_diagram(
    model_type = cfg["model_type"],
    value_function = cfg["value_function"],
    elicit = cfg["elicit"],
    noise = cfg["noise"],
    calculate_info_values= cfg["calculate_info_values"],
    ref_patient_chars = patient_chars,
    new_test = cfg["new_test"],
    logger = logger,
    output_dir = 'notebook/' + label,
)

2025-01-29 11:20:35,326 - INFO - Model type: linear
2025-01-29 11:20:35,327 - INFO - Reading network...
2025-01-29 11:20:35,341 - INFO - Calculating information values...
2025-01-29 11:21:28,505 - INFO - Eliciting value of comfort...
2025-01-29 11:21:28,506 - INFO - No elicitation of lambda values, taking default values...
2025-01-29 11:21:28,507 - INFO - Lambda values: [7.         4.01194248 6.80198452 4.01194248 6.80198452 4.01194248
 6.80198452 4.01194248 6.80198452 4.01194248 4.16848007 4.01194248
 4.16848007 4.01194248]
2025-01-29 11:21:28,507 - INFO - Saving network...
2025-01-29 11:21:28,556 - INFO - Plotting info functions...
2025-01-29 11:22:04,381 - INFO - Calculating final utilities...
2025-01-29 11:22:04,389 - INFO - Best info: 15.299634978175163
2025-01-29 11:22:04,390 - INFO - Worst info: 4.450147717014403e-308
2025-01-29 11:22:04,391 - INFO - PE info: 4.1
2025-01-29 11:22:04,392 - INFO - v_best: 104.06788025218766
2025-01-29 11:22:04,393 - INFO - v_worst: -3.910235286299

**Exogenous variable p = 0.02**

In [10]:
label = 'notebook_exogenous_var_002'
logger, log_dir = create_folders_logger(label = label, output_dir = 'notebook/' + label, date = False, time = False)

exog_char ={"Age": "age_2_young", 
    "Sex": "M",
    "SES": "ses_0",
    "SD": "SD_1_short",
    "PA": "PA_1",
    "Depression": False,
    "Smoking": "sm_1_not_smoker",
    "Anxiety": False,
    "BMI": "bmi_1_underweight",
    "Alcohol": "high",
    "Diabetes": False,
    "Hypertension": False,
    "Hyperchol_": False,
    }


update_influence_diagram(
    model_type = cfg["model_type"],
    value_function = cfg["value_function"],
    elicit = cfg["elicit"],
    noise = cfg["noise"],
    calculate_info_values= cfg["calculate_info_values"],
    ref_patient_chars = exog_char,
    new_test = cfg["new_test"],
    logger = logger,
    output_dir = 'notebook/' + label,
    exogenous_var_prob= 0.02
)

2025-01-29 11:22:04,618 - INFO - Model type: linear
2025-01-29 11:22:04,620 - INFO - Reading network...
2025-01-29 11:22:04,635 - INFO - Calculating information values...
2025-01-29 11:22:59,406 - INFO - Eliciting value of comfort...
2025-01-29 11:22:59,407 - INFO - No elicitation of lambda values, taking default values...
2025-01-29 11:22:59,408 - INFO - Lambda values: [7.         4.01194248 6.80198452 4.01194248 6.80198452 4.01194248
 6.80198452 4.01194248 6.80198452 4.01194248 4.16848007 4.01194248
 4.16848007 4.01194248]
2025-01-29 11:22:59,408 - INFO - Saving network...
2025-01-29 11:22:59,457 - INFO - Plotting info functions...
2025-01-29 11:23:34,385 - INFO - Calculating final utilities...
2025-01-29 11:23:34,392 - INFO - Best info: 15.299634978175163
2025-01-29 11:23:34,393 - INFO - Worst info: 4.450147717014403e-308
2025-01-29 11:23:34,394 - INFO - PE info: 4.1
2025-01-29 11:23:34,395 - INFO - v_best: 104.06788025218766
2025-01-29 11:23:34,396 - INFO - v_worst: -3.910235286299

**Exogenous variable p = 0.1**

In [11]:
label = 'notebook_exogenous_var_01'
logger, log_dir = create_folders_logger(label = label, output_dir = 'notebook/' + label, date = False, time = False)

exog_char ={"Age": "age_2_young", 
    "Sex": "M",
    "SES": "ses_0",
    "SD": "SD_1_short",
    "PA": "PA_1",
    "Depression": False,
    "Smoking": "sm_1_not_smoker",
    "Anxiety": False,
    "BMI": "bmi_1_underweight",
    "Alcohol": "high",
    "Diabetes": False,
    "Hypertension": False,
    "Hyperchol_": False,
    }

update_influence_diagram(
    model_type = cfg["model_type"],
    value_function = cfg["value_function"],
    elicit = cfg["elicit"],
    noise = cfg["noise"],
    calculate_info_values= cfg["calculate_info_values"],
    ref_patient_chars = exog_char,
    new_test = cfg["new_test"],
    logger = logger,
    output_dir = 'notebook/' + label,
    exogenous_var_prob= 0.1
)

2025-01-29 11:23:34,678 - INFO - Model type: linear
2025-01-29 11:23:34,679 - INFO - Reading network...
2025-01-29 11:23:34,703 - INFO - Calculating information values...
2025-01-29 11:24:27,584 - INFO - Eliciting value of comfort...
2025-01-29 11:24:27,585 - INFO - No elicitation of lambda values, taking default values...
2025-01-29 11:24:27,586 - INFO - Lambda values: [7.         4.01194248 6.80198452 4.01194248 6.80198452 4.01194248
 6.80198452 4.01194248 6.80198452 4.01194248 4.16848007 4.01194248
 4.16848007 4.01194248]
2025-01-29 11:24:27,587 - INFO - Saving network...
2025-01-29 11:24:27,632 - INFO - Plotting info functions...
2025-01-29 11:25:03,811 - INFO - Calculating final utilities...
2025-01-29 11:25:03,818 - INFO - Best info: 15.299634978175163
2025-01-29 11:25:03,819 - INFO - Worst info: 4.450147717014403e-308
2025-01-29 11:25:03,820 - INFO - PE info: 4.1
2025-01-29 11:25:03,821 - INFO - v_best: 104.06788025218766
2025-01-29 11:25:03,821 - INFO - v_worst: -3.910235286299

### Assessing the French strategy

**Patient 1**

In [12]:
label = 'notebook_4_2_patient_1'
logger, log_dir = create_folders_logger(label = label, output_dir = 'notebook/' + label, date = False, time = False)

patient_chars = {
    "Age": "age_5_old_adult", 
    "Sex": "M",
    "SD": "SD_2_normal",
    "PA": "PA_2",
    "Smoking": "sm_1_not_smoker",
    "BMI": "bmi_2_normal",
    "Alcohol": "low",
    "Diabetes": False,
    "Hypertension": False,
}

update_influence_diagram(
    model_type = cfg["model_type"],
    value_function = cfg["value_function"],
    elicit = cfg["elicit"],
    noise = cfg["noise"],
    calculate_info_values= cfg["calculate_info_values"],
    ref_patient_chars = patient_chars,
    new_test = cfg["new_test"],
    logger = logger,
    output_dir = 'notebook/' + label
)

2025-01-29 11:25:04,081 - INFO - Model type: linear
2025-01-29 11:25:04,083 - INFO - Reading network...
2025-01-29 11:25:04,098 - INFO - Calculating information values...
2025-01-29 11:25:57,952 - INFO - Eliciting value of comfort...
2025-01-29 11:25:57,953 - INFO - No elicitation of lambda values, taking default values...
2025-01-29 11:25:57,954 - INFO - Lambda values: [7.         4.01194248 6.80198452 4.01194248 6.80198452 4.01194248
 6.80198452 4.01194248 6.80198452 4.01194248 4.16848007 4.01194248
 4.16848007 4.01194248]
2025-01-29 11:25:57,955 - INFO - Saving network...
2025-01-29 11:25:57,991 - INFO - Plotting info functions...
2025-01-29 11:26:33,222 - INFO - Calculating final utilities...
2025-01-29 11:26:33,230 - INFO - Best info: 15.299634978175163
2025-01-29 11:26:33,232 - INFO - Worst info: 4.450147717014403e-308
2025-01-29 11:26:33,232 - INFO - PE info: 4.1
2025-01-29 11:26:33,233 - INFO - v_best: 104.06788025218766
2025-01-29 11:26:33,233 - INFO - v_worst: -3.910235286299

**Patient 2**

In [13]:
label = 'notebook_4_2_patient_2'
logger, log_dir = create_folders_logger(label = label, output_dir = 'notebook/' + label, date = False, time = False)

patient_chars = {
    "Age": "age_4_adult", 
    "Sex": "M",
    "SD": "SD_2_normal",
    "PA": "PA_2",
    "Smoking": "sm_1_not_smoker",
    "BMI": "bmi_2_normal",
    "Alcohol": "low",
    "Diabetes": True,
    "Hypertension": True,
}

update_influence_diagram(
    model_type = cfg["model_type"],
    value_function = cfg["value_function"],
    elicit = cfg["elicit"],
    noise = cfg["noise"],
    calculate_info_values= cfg["calculate_info_values"],
    ref_patient_chars = patient_chars,
    new_test = cfg["new_test"],
    logger = logger,
    output_dir = 'notebook/' + label
)

2025-01-29 11:26:33,359 - INFO - Model type: linear
2025-01-29 11:26:33,360 - INFO - Reading network...
2025-01-29 11:26:33,381 - INFO - Calculating information values...
2025-01-29 11:27:28,712 - INFO - Eliciting value of comfort...
2025-01-29 11:27:28,713 - INFO - No elicitation of lambda values, taking default values...
2025-01-29 11:27:28,714 - INFO - Lambda values: [7.         4.01194248 6.80198452 4.01194248 6.80198452 4.01194248
 6.80198452 4.01194248 6.80198452 4.01194248 4.16848007 4.01194248
 4.16848007 4.01194248]
2025-01-29 11:27:28,716 - INFO - Saving network...
2025-01-29 11:27:28,760 - INFO - Plotting info functions...
2025-01-29 11:28:05,739 - INFO - Calculating final utilities...
2025-01-29 11:28:05,746 - INFO - Best info: 15.299634978175163
2025-01-29 11:28:05,747 - INFO - Worst info: 4.450147717014403e-308
2025-01-29 11:28:05,749 - INFO - PE info: 4.1
2025-01-29 11:28:05,750 - INFO - v_best: 104.06788025218766
2025-01-29 11:28:05,750 - INFO - v_worst: -3.910235286299

**Patient 3**

In [14]:
label = 'notebook_4_2_patient_3'
logger, log_dir = create_folders_logger(label = label)

patient_chars = {
    "Age": "age_4_adult", 
    "Sex": "M",
    "SD": "SD_2_normal",
    "PA": "PA_1",
    "Smoking": "sm_3_ex_smoker",
    "BMI": "bmi_3_overweight",
    "Alcohol": "high",
}

update_influence_diagram(
    model_type = cfg["model_type"],
    value_function = cfg["value_function"],
    elicit = cfg["elicit"],
    noise = cfg["noise"],
    calculate_info_values= cfg["calculate_info_values"],
    ref_patient_chars = patient_chars,
    new_test = cfg["new_test"],
    logger = logger,
    output_dir = 'notebook/' + label
)

2025-01-29 11:28:05,962 - INFO - Model type: linear
2025-01-29 11:28:05,964 - INFO - Reading network...
2025-01-29 11:28:05,979 - INFO - Calculating information values...
2025-01-29 11:29:02,117 - INFO - Eliciting value of comfort...
2025-01-29 11:29:02,119 - INFO - No elicitation of lambda values, taking default values...
2025-01-29 11:29:02,120 - INFO - Lambda values: [7.         4.01194248 6.80198452 4.01194248 6.80198452 4.01194248
 6.80198452 4.01194248 6.80198452 4.01194248 4.16848007 4.01194248
 4.16848007 4.01194248]
2025-01-29 11:29:02,121 - INFO - Saving network...
2025-01-29 11:29:02,172 - INFO - Plotting info functions...
2025-01-29 11:29:38,408 - INFO - Calculating final utilities...
2025-01-29 11:29:38,417 - INFO - Best info: 15.299634978175163
2025-01-29 11:29:38,418 - INFO - Worst info: 4.450147717014403e-308
2025-01-29 11:29:38,419 - INFO - PE info: 4.1
2025-01-29 11:29:38,420 - INFO - v_best: 104.06788025218766
2025-01-29 11:29:38,420 - INFO - v_worst: -3.910235286299

### Designing a national personalised strategy

In [5]:
with open('config.yaml', 'r') as file:
    cfg = yaml.safe_load(file)

label = 'notebook_personalised_screening_str'
logger, log_dir = create_folders_logger(label=label, output_dir = 'notebook/' + label, date = False, time = False)

use_case_new_strategy(
    logger = logger,
    single_run=cfg['single_run'],
    output_dir = 'notebook/' + label,
    log_dir = log_dir,
    run_label = ''
)

2025-01-27 17:42:28,190 - INFO - Configuration variables of interest:
2025-01-27 17:42:28,191 - INFO - Single run: False
2025-01-27 17:42:28,192 - INFO - Number of runs: 8
2025-01-27 17:42:28,193 - INFO - Use all variables: True
2025-01-27 17:42:28,193 - INFO - Use case with new test: False
2025-01-27 17:42:28,194 - INFO - PE method: {'PE_cost': 5, 'PE_info': 4, 'PE_prob': 0.7}
2025-01-27 17:42:28,195 - INFO - Reading the network file...


2025-01-27 17:42:28,210 - INFO - Located at: decision_models/DM_screening_rel_point_cond_mut_info_linear.xdsl
2025-01-27 17:42:28,211 - INFO - Comfort values: 1 - 4.011942483017116, 2 - 4.168480067521723, 3 - 6.801984517973132, 4 - 7.0
2025-01-27 17:42:28,795 - INFO - All variables are kept in the dataframe for calculation of utilities.
2025-01-27 17:42:28,797 - INFO - Operational limits for the screening strategies: {'No_scr_no_col': inf, 'No_scr_col': 3000, 'gFOBT': 30000, 'FIT': 42000, 'Blood_based': 7000, 'Stool_DNA': 6000, 'CTC': 2000, 'Colon_capsule': 2000}
2025-01-27 17:42:28,828 - INFO - Multiple simulations of the tests will be performed...
Processing iterations: 100%|██████████| 8/8 [00:53<00:00,  6.73s/it]


{'': {'f1 old': 0.0,
  'f1 comp': 0.0,
  'old': 0.5155041748728112,
  'comp': 0.5160439521402076}}

### Benchmarking of new screening devices

In [18]:
cfg["patient_chars"]

{'Age': 'age_4_adult',
 'Sex': 'M',
 'SD': 'SD_2_normal',
 'PA': 'PA_2',
 'Smoking': 'sm_1_not_smoker',
 'BMI': 'bmi_2_normal',
 'Alcohol': 'low'}

In [19]:
label = 'notebook_benchmark_new_screening'
logger, log_dir = create_folders_logger(label=label, output_dir = 'notebook/' + label, date = False, time = False)

update_influence_diagram(
    model_type = cfg["model_type"],
    value_function = cfg["value_function"],
    elicit = cfg["elicit"],
    noise = cfg["noise"],
    calculate_info_values= cfg["calculate_info_values"],
    ref_patient_chars = cfg["patient_chars"],
    new_test = True,
    logger = logger,
    output_dir = 'notebook/' + label
)

2025-01-29 11:50:13,152 - INFO - Model type: linear
2025-01-29 11:50:13,153 - INFO - Reading network...
2025-01-29 11:50:13,172 - INFO - Calculating information values...
2025-01-29 11:51:16,288 - INFO - Eliciting value of comfort...
2025-01-29 11:51:16,290 - INFO - No elicitation of lambda values, taking default values...
2025-01-29 11:51:16,291 - INFO - Lambda values: [7.         4.01194248 6.80198452 4.01194248 6.80198452 4.01194248
 6.80198452 4.01194248 6.80198452 4.01194248 4.16848007 4.01194248
 4.16848007 4.01194248]
2025-01-29 11:51:16,292 - INFO - Adding new test values...
2025-01-29 11:52:30,530 - INFO - Saving network...
2025-01-29 11:52:30,593 - INFO - Plotting info functions...
2025-01-29 11:53:54,369 - INFO - Calculating final utilities...
2025-01-29 11:53:54,384 - INFO - Best info: 15.299634978175163
2025-01-29 11:53:54,385 - INFO - Worst info: 4.450147717014403e-308
2025-01-29 11:53:54,385 - INFO - PE info: 4.1
2025-01-29 11:53:54,386 - INFO - v_best: 104.0678802521876

In [20]:
cfg["lambda_list"]

[4.8, 5, 6.801984517973132, 7.0]

In [21]:
label = 'notebook_new_test_screening_strategy'
logger, log_dir = create_folders_logger(label=label, output_dir = 'notebook/' + label, date = False, time = False)

use_case_new_strategy(
    logger = logger,
    use_case_new_test = True,
    single_run=cfg['single_run'],
    output_dir = 'notebook/' + label,
    log_dir = log_dir,
    run_label = '',
)

2025-01-29 11:53:54,539 - INFO - Configuration variables of interest:
2025-01-29 11:53:54,540 - INFO - Single run: False
2025-01-29 11:53:54,540 - INFO - Number of runs: 200
2025-01-29 11:53:54,541 - INFO - Use all variables: True
2025-01-29 11:53:54,542 - INFO - Use case with new test: True
2025-01-29 11:53:54,542 - INFO - Reading the network file...
2025-01-29 11:53:54,578 - INFO - Located at: decision_models/DM_screening_rel_point_cond_mut_info_linear_new_test.xdsl
2025-01-29 11:53:54,579 - INFO - Comfort values: 1 - 4.011942483017116, 2 - 4.168480067521723, 3 - 6.801984517973132, 4 - 7.0
2025-01-29 11:53:55,194 - INFO - All variables are kept in the dataframe for calculation of utilities.
2025-01-29 11:53:55,196 - INFO - Operational limits for the screening strategies: {'No_scr_no_col': inf, 'No_scr_col': 3000, 'gFOBT': 30000, 'FIT': 42000, 'Blood_based': 7000, 'Stool_DNA': 5000, 'CTC': 2000, 'Colon_capsule': 2000, 'New_test': 50000}
2025-01-29 11:53:55,232 - INFO - Multiple simula

{'new_test': {'f1 old': 0.49952347525311747, 'f1 comp': 0.5037039102606254}}

## Sensitivity analysis

### Sensitivity analysis for the PE

In [5]:
label = 'notebook_sens_analysis_PE_method'
logger, log_dir = create_folders_logger(label=label, output_dir = 'notebook/' + label, date = False, time = False)

sens_analysis_PE_method(
    label = label,
    output_dir = 'notebook/' + label,
    log_dir = log_dir,  
)

2025-01-27 17:50:51,100 - INFO - Reading from file: decision_models/DM_screening_rel_point_cond_mut_info_linear.xdsl
2025-01-27 17:50:51,100 - INFO - Reading from file: decision_models/DM_screening_rel_point_cond_mut_info_linear.xdsl
2025-01-27 17:50:51,102 - INFO - PE_info_array: [4.2 4.  3.8]
2025-01-27 17:50:51,102 - INFO - PE_info_array: [4.2 4.  3.8]
2025-01-27 17:50:51,103 - INFO - PE_cost_array: [ 5 10 50]
2025-01-27 17:50:51,103 - INFO - PE_cost_array: [ 5 10 50]
2025-01-27 17:50:54,889 - INFO - Params: [1.016419338142636, 0.8801528395772084, 0.03681253893587941]
2025-01-27 17:50:54,889 - INFO - Params: [1.016419338142636, 0.8801528395772084, 0.03681253893587941]
2025-01-27 17:51:59,901 - INFO - PE_info: 4.2, PE_cost: 5, DONE!
2025-01-27 17:51:59,901 - INFO - PE_info: 4.2, PE_cost: 5, DONE!
2025-01-27 17:52:00,603 - INFO - Params: [1.0260096777692866, 0.889649200484113, 0.036469730286183666]
2025-01-27 17:52:00,603 - INFO - Params: [1.0260096777692866, 0.889649200484113, 0.0364

### Sensitivity analysis for comfort parameters

In [6]:
label = 'notebook_sens_analysis_comf_param'
logger, log_dir = create_folders_logger(label = label, output_dir= 'notebook/' + label, date = False, time = False)

sens_analysis_elicitation(
    label = label,
    output_dir = 'notebook/' + label
)

2025-01-29 13:04:17,390 - INFO - Configuration variables of interest:
2025-01-29 13:04:17,390 - INFO - Configuration variables of interest:
2025-01-29 13:04:17,391 - INFO - Single run: False
2025-01-29 13:04:17,391 - INFO - Single run: False
2025-01-29 13:04:17,393 - INFO - Use all variables: True
2025-01-29 13:04:17,393 - INFO - Use all variables: True
2025-01-29 13:04:17,394 - INFO - Use case with new test: False
2025-01-29 13:04:17,394 - INFO - Use case with new test: False
2025-01-29 13:04:17,395 - INFO - PE method: {'PE_cost': 50, 'PE_info': 4.1, 'PE_prob': 0.7}
2025-01-29 13:04:17,395 - INFO - PE method: {'PE_cost': 50, 'PE_info': 4.1, 'PE_prob': 0.7}
2025-01-29 13:04:17,396 - INFO - Change lambdas? True
2025-01-29 13:04:17,396 - INFO - Change lambdas? True
2025-01-29 13:04:17,397 - INFO - Read lambda list from config? True
2025-01-29 13:04:17,397 - INFO - Read lambda list from config? True
2025-01-29 13:04:17,398 - INFO - Lambdas: [4.011942483017116, 4.168480067521723, 6.8019845

Counts: best_option
No_scr_no_col    288682
No_scr_col            0
gFOBT                 0
FIT                3201
Blood_based           0
Stool_DNA         47824
CTC                   0
Colon_capsule         0
Name: count, dtype: int64


In [7]:
label = 'notebook_sens_analysis_comf_param'
logger, log_dir = create_folders_logger(label = label, output_dir= 'notebook/' + label, date = False, time = False)

lambda_list = cfg["lambda_list"].copy()
lambda_list[2] = 6.3

sens_analysis_elicitation(
    lambda_list = lambda_list,
    label = label,
    output_dir = 'notebook/' + label
)

2025-01-29 13:07:20,934 - INFO - Configuration variables of interest:
2025-01-29 13:07:20,934 - INFO - Configuration variables of interest:
2025-01-29 13:07:20,934 - INFO - Configuration variables of interest:
2025-01-29 13:07:20,937 - INFO - Single run: False
2025-01-29 13:07:20,937 - INFO - Single run: False
2025-01-29 13:07:20,937 - INFO - Single run: False
2025-01-29 13:07:20,938 - INFO - Use all variables: True
2025-01-29 13:07:20,938 - INFO - Use all variables: True
2025-01-29 13:07:20,938 - INFO - Use all variables: True
2025-01-29 13:07:20,940 - INFO - Use case with new test: False
2025-01-29 13:07:20,940 - INFO - Use case with new test: False
2025-01-29 13:07:20,940 - INFO - Use case with new test: False
2025-01-29 13:07:20,941 - INFO - PE method: {'PE_cost': 50, 'PE_info': 4.1, 'PE_prob': 0.7}
2025-01-29 13:07:20,941 - INFO - PE method: {'PE_cost': 50, 'PE_info': 4.1, 'PE_prob': 0.7}
2025-01-29 13:07:20,941 - INFO - PE method: {'PE_cost': 50, 'PE_info': 4.1, 'PE_prob': 0.7}
2

Counts: best_option
No_scr_no_col    313960
No_scr_col            0
gFOBT                 0
FIT                   0
Blood_based           0
Stool_DNA         25747
CTC                   0
Colon_capsule         0
Name: count, dtype: int64


In [8]:
label = 'notebook_sens_analysis_comf_param'
logger, log_dir = create_folders_logger(label = label, output_dir= 'notebook/' + label, date = False, time = False)

lambda_list = cfg["lambda_list"]
lambda_list[0] = 4.8
lambda_list[1] = 5

sens_analysis_elicitation(
    lambda_list = lambda_list,
    label = label,
    output_dir = 'notebook/' + label
)

2025-01-29 13:10:02,223 - INFO - Configuration variables of interest:
2025-01-29 13:10:02,223 - INFO - Configuration variables of interest:
2025-01-29 13:10:02,226 - INFO - Single run: False
2025-01-29 13:10:02,226 - INFO - Single run: False
2025-01-29 13:10:02,228 - INFO - Use all variables: True
2025-01-29 13:10:02,228 - INFO - Use all variables: True
2025-01-29 13:10:02,230 - INFO - Use case with new test: False
2025-01-29 13:10:02,230 - INFO - Use case with new test: False
2025-01-29 13:10:02,231 - INFO - PE method: {'PE_cost': 50, 'PE_info': 4.1, 'PE_prob': 0.7}
2025-01-29 13:10:02,231 - INFO - PE method: {'PE_cost': 50, 'PE_info': 4.1, 'PE_prob': 0.7}
2025-01-29 13:10:02,233 - INFO - Change lambdas? True
2025-01-29 13:10:02,233 - INFO - Change lambdas? True
2025-01-29 13:10:02,234 - INFO - Read lambda list from config? True
2025-01-29 13:10:02,234 - INFO - Read lambda list from config? True
2025-01-29 13:10:02,235 - INFO - Lambdas: [4.8, 5, 6.801984517973132, 7.0]
2025-01-29 13:1

Counts: best_option
No_scr_no_col    179840
No_scr_col            0
gFOBT                 0
FIT                   0
Blood_based           0
Stool_DNA        159867
CTC                   0
Colon_capsule         0
Name: count, dtype: int64
